In [1]:
import pandas as pd

# Data Inflasi

In [2]:
df = pd.read_excel('data/Data Inflasi (2000-2025).xlsx')

In [3]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,Data Inflasi,Data Inflasi,Data Inflasi,Data Inflasi
2,NaN,NaN,NaN,NaN
3,No,Periode,Data Inflasi,NaN
4,1,Maret 2025,1.03 %,NaN
...,...,...,...,...
267,264,April 2003,7.62 %,NaN
268,265,Maret 2003,7.17 %,NaN
269,266,Februari 2003,7.6 %,NaN
270,267,Januari 2003,8.68 %,NaN


In [4]:
df = df.iloc[4:].reset_index(drop=True).drop(columns=['Unnamed: 0', 'Unnamed: 3'])

In [5]:
df = df.rename(columns={'Unnamed: 1': 'Tahun', 'Unnamed: 2': 'Inflation Percentage'})

In [6]:
df

,Tahun,Inflation Percentage
0,Maret 2025,1.03 %
1,Februari 2025,-0.09 %
2,Januari 2025,0.76 %
3,Desember 2024,1.57 %
4,November 2024,1.55 %
...,...,...
263,April 2003,7.62 %
264,Maret 2003,7.17 %
265,Februari 2003,7.6 %
266,Januari 2003,8.68 %


In [7]:
# Split the Tahun column into Bulan and Tahun
df[['Bulan', 'Tahun']] = df['Tahun'].str.split(' ', expand=True)

In [8]:
df

,Tahun,Inflation Percentage,Bulan
0,2025,1.03 %,Maret
1,2025,-0.09 %,Februari
2,2025,0.76 %,Januari
3,2024,1.57 %,Desember
4,2024,1.55 %,November
...,...,...,...
263,2003,7.62 %,April
264,2003,7.17 %,Maret
265,2003,7.6 %,Februari
266,2003,8.68 %,Januari


In [9]:
df.to_csv('data-cleaned/inflation_data_cleaned.csv')

# Garis Kemiskinan Makanan

In [10]:
data_makanan = {}

for tahun in range(2015, 2025):
    file = f"data/Garis Kemiskinan Makanan (Rupiah_Kapita_Bulan) Menurut Provinsi dan Daerah, {tahun}.csv"
    df_makanan = pd.read_csv(file)
    data_makanan[tahun] = df_makanan

In [11]:
data_makanan[2015].head(10)

,38 Provinsi,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,Garis Kemiskinan Makanan (Rupiah/Kapita/Bulan)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Perkotaan,NaN,NaN,Perdesaan,NaN,NaN,Perkotaan+Perdesaan,NaN,NaN
2,NaN,2015,NaN,NaN,2015,NaN,NaN,2015,NaN,NaN
3,NaN,Semester 1 (Maret),Semester 2 (September),Tahunan,Semester 1 (Maret),Semester 2 (September),Tahunan,Semester 1 (Maret),Semester 2 (September),Tahunan
4,ACEH,293697,302128,-,297479,306737,-,296406,305428,-
5,SUMATERA UTARA,261677,273271,-,266143,284650,-,263932,279015,-
6,SUMATERA BARAT,288410,301356,-,293768,313294,-,291641,308554,-
7,RIAU,280361,288956,-,302422,318195,-,293851,306835,-
8,JAMBI,297507,308379,-,247652,258946,-,262791,273957,-
9,SUMATERA SELATAN,263734,272915,-,243702,254209,-,250850,260885,-


In [12]:
for tahun in range(2015, 2025):
    data_makanan[tahun] = data_makanan[tahun].iloc[4:].reset_index(drop=True).drop(columns=['Unnamed: 3','Unnamed: 6','Unnamed: 9'])
    data_makanan[tahun] = data_makanan[tahun].rename(columns={'Unnamed: 1': 'Perkotaan Semester 1 (Maret)', 'Unnamed: 2': 'Perkotaan Semester 2 (September)',
                                                             'Unammed: 4':'Perdesaan Semester 1 (Maret)', 'Unnamed: 5':'Perdesaan Semester 2 (September)',
                                                             'Unnamed: 7':'Perkotaan+Perdesaan Semester 1 (Maret)', 'Unnamed: 8':'Perkotaan+Perdesaan Semester 2 (September)'})
    data_makanan[tahun]['Tahun'] = tahun

In [14]:
data_makanan[2024].head(10)

,38 Provinsi,Perkotaan Semester 1 (Maret),Perkotaan Semester 2 (September),Unnamed: 4,Perdesaan Semester 2 (September),Perkotaan+Perdesaan Semester 1 (Maret),Perkotaan+Perdesaan Semester 2 (September),Tahun
0,ACEH,519288,528266,492730,494778,501403,506089,2024
1,SUMATERA UTARA,502288,508995,476787,479327,490540,495730,2024
2,SUMATERA BARAT,532696,540096,542700,545435,537985,542853,2024
3,RIAU,529025,533936,501450,505622,511959,516772,2024
4,JAMBI,561284,569981,459892,464125,491690,497637,2024
5,SUMATERA SELATAN,433537,447563,402078,409879,413237,423507,2024
6,BENGKULU,546830,551184,468390,469716,492988,493660,2024
7,LAMPUNG,474260,486443,421189,430504,438079,448211,2024
8,KEP. BANGKA BELITUNG,643357,650379,690751,698232,663408,670142,2024
9,KEP. RIAU,532365,551982,566093,573728,535976,554329,2024


In [24]:
Data_makanan = pd.concat([data_makanan[tahun] for tahun in range(2015, 2025)],ignore_index = True)
Data_makanan

,38 Provinsi,Perkotaan Semester 1 (Maret),Perkotaan Semester 2 (September),Unnamed: 4,Perdesaan Semester 2 (September),Perkotaan+Perdesaan Semester 1 (Maret),Perkotaan+Perdesaan Semester 2 (September),Tahun
0,ACEH,293697,302128,297479,306737,296406,305428,2015
1,SUMATERA UTARA,261677,273271,266143,284650,263932,279015,2015
2,SUMATERA BARAT,288410,301356,293768,313294,291641,308554,2015
3,RIAU,280361,288956,302422,318195,293851,306835,2015
4,JAMBI,297507,308379,247652,258946,262791,273957,2015
...,...,...,...,...,...,...,...,...
385,PAPUA,507129,533022,404318,413604,462491,484209,2024
386,PAPUA SELATAN,467204,502110,340771,351258,390999,418637,2024
387,PAPUA TENGAH,607350,638860,579784,606795,586166,615022,2024
388,PAPUA PEGUNUNGAN,990068,1065131,746176,803603,757648,820343,2024


In [22]:
Data_makanan.to_csv('data-cleaned/Garis_Kemiskinan_Makanan_cleaned.csv')

# Garis Kemiskinan Non-Makanan

In [16]:
data_nonmakanan = {}

for tahun in range(2015, 2025):
    file = f"data/Garis Kemiskinan Non-Makanan (Rupiah_Kapita_Bulan) Menurut Provinsi dan Daerah, {tahun}.csv"
    df_nonmakanan = pd.read_csv(file)
    data_nonmakanan[tahun] = df_nonmakanan

In [17]:
data_nonmakanan[2015].head(10)

,38 Provinsi,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,Garis Kemiskinan Non-Makanan (Rupiah/Kapita/Bu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Perkotaan,NaN,NaN,Perdesaan,NaN,NaN,Perkotaan+Perdesaan,NaN,NaN
2,NaN,2015,NaN,NaN,2015,NaN,NaN,2015,NaN,NaN
3,NaN,Semester 1 (Maret),Semester 2 (September),Tahunan,Semester 1 (Maret),Semester 2 (September),Tahunan,Semester 1 (Maret),Semester 2 (September),Tahunan
4,ACEH,116717,118196,-,84638,87682,-,93744,96344,-
5,SUMATERA UTARA,102643,106627,-,65752,67987,-,84021,87123,-
6,SUMATERA BARAT,117925,121984,-,75985,77884,-,92637,95393,-
7,RIAU,124441,128812,-,93237,98585,-,105361,110329,-
8,JAMBI,108566,115476,-,69186,70949,-,81144,84470,-
9,SUMATERA SELATAN,101603,105823,-,61880,65785,-,76055,80073,-


In [18]:
for tahun in range(2015, 2025):
    data_nonmakanan[tahun] = data_nonmakanan[tahun].iloc[4:].reset_index(drop=True).drop(columns=['Unnamed: 3','Unnamed: 6','Unnamed: 9'])
    data_nonmakanan[tahun] = data_nonmakanan[tahun].rename(columns={'Unnamed: 1': 'Perkotaan Semester 1 (Maret)', 'Unnamed: 2': 'Perkotaan Semester 2 (September)',
                                                             'Unammed: 4':'Perdesaan Semester 1 (Maret)', 'Unnamed: 5':'Perdesaan Semester 2 (September)',
                                                             'Unnamed: 7':'Perkotaan+Perdesaan Semester 1 (Maret)', 'Unnamed: 8':'Perkotaan+Perdesaan Semester 2 (September)'})
    data_nonmakanan[tahun]['Tahun'] = tahun

In [19]:
data_nonmakanan[2024].head(10)

,38 Provinsi,Perkotaan Semester 1 (Maret),Perkotaan Semester 2 (September),Unnamed: 4,Perdesaan Semester 2 (September),Perkotaan+Perdesaan Semester 1 (Maret),Perkotaan+Perdesaan Semester 2 (September),Tahun
0,ACEH,176760,175934,150668,150909,159824,159766,2024
1,SUMATERA UTARA,165634,165432,133477,135118,151883,152606,2024
2,SUMATERA BARAT,190871,191139,149201,151617,170431,172138,2024
3,RIAU,211197,212540,167924,167809,185337,185848,2024
4,JAMBI,182760,184254,146258,148620,158425,160463,2024
5,SUMATERA SELATAN,156059,155742,131714,131942,140960,140955,2024
6,BENGKULU,189928,191581,172095,172904,178107,179156,2024
7,LAMPUNG,166648,169210,139174,141298,148472,150807,2024
8,KEP. BANGKA BELITUNG,248812,252943,239627,239769,244989,247531,2024
9,KEP. RIAU,258987,260312,183511,191157,251235,253273,2024


In [25]:
Data_nonmakanan = pd.concat([data_nonmakanan[tahun] for tahun in range(2015, 2025)],ignore_index = True)
Data_nonmakanan

,38 Provinsi,Perkotaan Semester 1 (Maret),Perkotaan Semester 2 (September),Unnamed: 4,Perdesaan Semester 2 (September),Perkotaan+Perdesaan Semester 1 (Maret),Perkotaan+Perdesaan Semester 2 (September),Tahun
0,ACEH,116717,118196,84638,87682,93744,96344,2015
1,SUMATERA UTARA,102643,106627,65752,67987,84021,87123,2015
2,SUMATERA BARAT,117925,121984,75985,77884,92637,95393,2015
3,RIAU,124441,128812,93237,98585,105361,110329,2015
4,JAMBI,108566,115476,69186,70949,81144,84470,2015
...,...,...,...,...,...,...,...,...
385,PAPUA,242545,254161,165977,179865,211880,224602,2024
386,PAPUA SELATAN,178832,179426,101319,102833,128980,130009,2024
387,PAPUA TENGAH,312754,320179,134486,138735,177949,183311,2024
388,PAPUA PEGUNUNGAN,293852,312333,246198,255094,249412,258817,2024


In [26]:
Data_nonmakanan.to_csv('data-cleaned/Garis_Kemiskinan_Non-Makanan_cleaned.csv')